#  LangChain의 RAG 콤포넌트 
- 문서 로더 (Document Loader) 

### **학습 목표:**  문서 로더와 텍스트 분할기를 적절히 활용할 수 있다

### **실습 자료**: 

- data/transformer.pdf
- data/kakao_chat.json
- data/kakao_chat.jsonl
- data/kbo_teams_2023.csv
- data/리비안_KR.txt
- data/테슬라_KR.txt
- articles/notionai.pdf

---

# 환경 설정 및 준비

`(1) Env 환경변수`

In [ ]:
from dotenv import load_dotenv
load_dotenv()

`(2) 기본 라이브러리`

In [2]:
import os
from glob import glob

from pprint import pprint
import json

# 다양한 문서 형식 처리하기

- 역할: Document Loader는 다양한 소스에서 문서를 로드
- 구현: 
    - Document Loader는 BaseLoader 인터페이스를 통해서 구현
    - `.load()` 또는 `.lazy_load()` 메서드를 통해 동일한 방식으로 호출 
    - 대용량 데이터셋의 경우 메모리 효율을 위해 `.lazy_load()`를 사용하는 것을 권장 
- 종류:
    - PDF 파일 로더
    - 웹 페이지 로더 
    - CSV 데이터 로더
    - 디렉토리 로더
    - HTML 데이터 로더
    - JSON 데이터 로더
    - Markdown 데이터 로더
    - Microsoft Office 데이터 로더

### 1. **PDF 파일 로더**

- uv add pypdf 또는 pip install pypdf


In [ ]:
from langchain_community.document_loaders import PyPDFLoader

# PDF 로더 초기화
pdf_loader = PyPDFLoader('./data/transformer.pdf')

# 동기 로딩
pdf_docs = pdf_loader.load()
print(f'PDF 문서 개수: {len(pdf_docs)}')

In [ ]:
# 첫 번째 문서의 내용 출력
print(pdf_docs[0].page_content)

In [ ]:
# 첫 번째 문서의 메타데이터 출력
print(pdf_docs[0].metadata)

In [ ]:
# 비동기 로딩 (대용량 파일에 권장)
async for page in pdf_loader.alazy_load():
    # 페이지별 처리
    print(page.metadata)  # 메타데이터 출력
    print(page.page_content) # 페이지 내용 출력
    print("-"*80)

### 2. **웹 문서 로더**


In [ ]:
from langchain_community.document_loaders import WebBaseLoader

# 기본적인 텍스트 추출
web_loader = WebBaseLoader(
    web_paths=[
        "https://python.langchain.com/", 
        "https://js.langchain.com/",
        ],
    )

# 동기 로딩
web_docs = web_loader.load()

len(web_docs)

In [ ]:
web_docs[0].metadata

In [ ]:
print(web_docs[0].page_content)

In [ ]:
# 특정 HTML 요소만 파싱하고 싶을 경우 (bs4 활용)
import bs4

web_loader = WebBaseLoader(
    web_paths=["https://python.langchain.com/"],
    bs_kwargs={
        "parse_only": bs4.SoupStrainer(class_="theme-doc-markdown markdown"),
    },
    bs_get_text_kwargs={
        "separator": " | ",    # 구분자
        "strip": True          # 공백 제거
    }
)

# 동기 로딩
web_docs = web_loader.load()

len(web_docs)

In [ ]:
print(web_docs[0].page_content)

### 3. **JSON 파일 로더**

- 설치: pip install jq 또는 uv add jq


In [ ]:
from langchain_community.document_loaders import JSONLoader 

json_loader = JSONLoader(
    file_path="./data/kakao_chat.json",
    jq_schema=".messages[].content",    # messages 배열의 content 필드만 추출
    text_content=True,                  # 추출하려는 필드가 텍스트인지 여부
)

json_docs = json_loader.load()

print("문서의 수:", len(json_docs))
print("-" * 50)
print("처음 문서의 메타데이터: \n", json_docs[0].metadata['seq_num'])
print("-" * 50)
print("처음 문서의 내용: \n", json_docs[0].page_content)

In [ ]:
from langchain_community.document_loaders import JSONLoader

json_loader = JSONLoader(
    file_path="./data/kakao_chat.json",
    jq_schema=".messages[]",     # messages 배열의 모든 아이템을 추출
    text_content=False,          # 추출하려는 필드가 텍스트인지 여부
)

json_docs = json_loader.load()

print("문서의 수:", len(json_docs))
print("-" * 50)
print("처음 문서의 메타데이터: \n", json_docs[0].metadata['seq_num'])
print("-" * 50)
print("처음 문서의 내용: \n", json_docs[0].page_content)

In [ ]:
# 실제 입력 데이터 확인
print(json_docs[4].page_content)  # 실제 JSON 문자열 확인

In [ ]:
# 유니코드 디코딩 (한글 문자들이 유니코드 이스케이프 시퀀스로 인코딩되어 있음)
from langchain_core.documents import Document

decoded_json_docs = []
for doc in json_docs:

    decoded_data = json.loads(doc.page_content)
    document_obj = Document(page_content=json.dumps(decoded_data, ensure_ascii=False), metadata=doc.metadata)
    decoded_json_docs.append(document_obj)

print("문서의 수:", len(decoded_json_docs))
print("-" * 50)
print("처음 문서의 메타데이터: \n", decoded_json_docs[0].metadata['seq_num'])
print("-" * 50)
print("처음 문서의 내용: \n", decoded_json_docs[0].page_content)


In [ ]:
# JSONL 파일 로드하기
json_loader = JSONLoader(
    file_path="./data/kakao_chat.jsonl",
    jq_schema=".content",
    json_lines=True,
)

json_docs = json_loader.load()

print("문서의 수:", len(json_docs))
print("-" * 50)
print("처음 문서의 메타데이터: \n", json_docs[0].metadata['seq_num'])
print("-" * 50)
print("처음 문서의 내용: \n", json_docs[0].page_content)


In [ ]:
# JSONL 파일 로드하기
json_loader = JSONLoader(
    file_path="./data/kakao_chat.jsonl",
    jq_schema=".",
    content_key="content",
    json_lines=True,
)

json_docs = json_loader.load()

print("문서의 수:", len(json_docs))
print("-" * 50)
print("처음 문서의 메타데이터: \n", json_docs[0].metadata['seq_num'])
print("-" * 50)
print("처음 문서의 내용: \n", json_docs[0].page_content)


### 4. **CSV 파일 로더**


In [ ]:
from langchain_community.document_loaders.csv_loader import CSVLoader

# 기본 파일 로드
csv_loader = CSVLoader("./data/kbo_teams_2023.csv")
csv_docs = csv_loader.load()

print("문서의 수:", len(csv_docs))
print("-" * 50)
print("처음 문서의 메타데이터: \n", csv_docs[0].metadata)
print("-" * 50)
print("처음 문서의 내용: \n", csv_docs[0].page_content)

In [ ]:
## CSV 파싱 커스터마이징

csv_loader = CSVLoader(
    file_path="./data/kbo_teams_2023.csv",
    csv_args={
        "delimiter": ",",               # 구분자 지정
        "quotechar": '"',               # 따옴표 문자 지정
    }
)

csv_docs = csv_loader.load()

print("문서의 수:", len(csv_docs))
print("-" * 50)
print("처음 문서의 메타데이터: \n", csv_docs[0].metadata)
print("-" * 50)
print("처음 문서의 내용: \n", csv_docs[0].page_content)

In [ ]:
## 소스 컬럼 지정

csv_loader = CSVLoader(
    file_path="./data/kbo_teams_2023.csv",
    source_column="Team"  # 이 컬럼의 값이 메타데이터의 source로 사용됨
)

csv_docs = csv_loader.load()

print("문서의 수:", len(csv_docs))
print("-" * 50)
print("처음 문서의 메타데이터: \n", csv_docs[0].metadata)
print("-" * 50)
print("처음 문서의 내용: \n", csv_docs[0].page_content)

### 5. **Directory 로더**
- 개요: 디렉토리에서 파일들을 읽어 Document 객체로 변환
- 기능: 
    1. 파일시스템 탐색 (와일드카드 패턴 지원)
    2. 멀티스레딩을 통한 파일 I/O
    3. 인코딩 오류 등의 예외 처리

In [ ]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import TextLoader

# 기본 사용법
dir_loader = DirectoryLoader(
    path="./",              # 파일 경로 - 현재 디렉토리
    glob="**/*_KR.txt",     # 파일 확장자 - txt 파일만 로드
    loader_cls=TextLoader,  # TextLoader, CSVLoader, UnstructuredFileLoader 등 지원 
    )
dir_docs = dir_loader.load()

print("문서의 수:", len(dir_docs))
print("-" * 50)
print("처음 문서의 메타데이터: \n", dir_docs[0].metadata)
print("-" * 50)
print("처음 문서의 내용: \n", dir_docs[0].page_content)

In [ ]:
## 진행 상태바 표시
dir_loader = DirectoryLoader(
    path="./",              # 파일 경로 - 현재 디렉토리
    glob="**/*_KR.txt",     # 파일 확장자 - txt 파일만 로드
    loader_cls=TextLoader,  # TextLoader, CSVLoader, UnstructuredFileLoader 등 지원 
    show_progress=True,     # 진행 상태바 표시
    )
dir_docs = dir_loader.load()

# [실습 프로젝트]

- PDF 파일(articles/notionai.pdf)을 로드합니다. 
- 로드된 문서의 개수와 각 문서의 텍스트 길이를 출력합니다. 

In [ ]:
# 여기에 코드를 작성하세요.